In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# init the constants and initial p, q
N = 30
k = 50.
g = 9.81
y0 = 40.
#m = np.ones(N)
xs = np.arange(N)
ys = np.array([y0] * N)
q0 = np.vstack((xs, ys)).T
p0 = np.zeros((N, 2))

In [ ]:
# for energies and gradient calc
# forget about the ground
'''def gravity(y):
    if(y >= 0):
        return g*y
    else:
        return 1e20'''
def gravity(y):
    """Calculate the gravity potential for unit mass at height y."""
    return g*y

def lengths(xs, ys):
    """Return the rope lengths as an nparray of length (n-1)."""
    xdiff = xs[1:] - xs[:-1]
    ydiff = ys[1:] - ys[:-1]
    return np.sqrt(np.square(xdiff) + np.square(ydiff))

def potential(q, m = 1):
    """Return the potential of the system."""
    xs, ys = q[:, 0], q[:, 1]
    ls = lengths(xs, ys)
    return (np.vectorize(gravity)(ys) * m).sum() + k * np.square(ls - 1).sum()

def kinetic(p, m = 1):
    """Return the kinetic energy of the system."""
    return (np.square(p)/m).sum() / 2

# gradients
def potential_grad(q, m = 1):
    """
    Return the gradient of the potential of system,
        with fixed end balls (by setting F on them to 0).
    """
    if(np.shape(m) == ()):
        m = m * np.ones(N)
    xs, ys = q[:, 0], q[:, 1]
    grad = np.zeros((N, 2))
    ls = lengths(xs, ys)
    delta_ls = ls - 1
    xdiff = xs[1:] - xs[:-1]
    ydiff = ys[1:] - ys[:-1]
    grad[1:-1, 0] = 2 * k * delta_ls[:-1] * xdiff[:-1] / ls[:-1] - 2 * k * delta_ls[1:] * xdiff[1:] / ls[1:]
    grad[1:-1, 1] = 2 * k * delta_ls[:-1] * ydiff[:-1] / ls[:-1] - 2 * k * delta_ls[1:] * ydiff[1:] / ls[1:] + m[1:-1] * g
    #for i in range(1, 29):
    #    grad[i, 0] = 2*k*(ls[i-1] - 1)*(xs[i]-xs[i-1])/ls[i-1] + 2*k*(ls[i] - 1)*(xs[i]-xs[i+1])/ls[i]
    #    grad[i, 1] = m[i] * g + 2*k*(ls[i-1] - 1)*(ys[i]-ys[i-1])/ls[i-1] + 2*k*(ls[i] - 1)*(ys[i]-ys[i+1])/ls[i]
    return grad

In [ ]:
def vv(potenital_grad, steps, q0, p0, m = 1, timestep = 0.01):
    """Velocity Verlet for autonomous system (pot_grad only a function of q)."""
    halfstep = timestep / 2
    vec_shape = (steps + 1, q0.shape[0], q0.shape[1])
    qs = np.zeros(vec_shape)
    ps = np.zeros(vec_shape)
    qs[0], ps[0] = q0, p0
    potg = potenital_grad(qs[0])
    for i in range(1, steps + 1):
        p_temp = ps[i - 1] - potg * halfstep
        qs[i] = qs[i - 1] + p_temp / m * timestep
        potg = potenital_grad(qs[i])
        ps[i] = p_temp - potg * halfstep
    return qs, ps

In [ ]:
size = 2000
qx, px = vv(potential_grad, size, q0, p0, timestep = 0.005)

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion()
fig.show()
fig.canvas.draw()
ax.set_ylim([0., y0 + 2])
ax.set_xlabel('x / m')
ax.set_ylabel('y / m')
scat = ax.scatter(qx[0, :, 0], qx[0, :, 1])
for i in range(1, 500):
    scat.remove()    
    scat = ax.scatter(qx[4 * i, :, 0], qx[4 * i, :, 1], color = 'b')
    fig.canvas.draw()

In [ ]:
# Plot the energies in every step
E_p = np.zeros(size + 1)
E_k = np.zeros(size + 1)
for i in range(size + 1):
    E_p[i], E_k[i] = potential(qx[i]), kinetic(px[i])
E = E_p + E_k
times = 0.01 * np.arange(size + 1)
%matplotlib inline
plt.plot(times, E_p, color = 'm', label = 'potential')
plt.plot(times, E_k, color = 'g', label = 'kinetic')
plt.plot(times, E, color = 'b', label = 'total')
plt.ylabel('Energy / a.u.')
plt.xlabel('time / s')
plt.legend()